In [22]:
import pandas as pd
import numpy as np

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

In [23]:
merged = pd.read_csv(path+'data/merged.csv')

In [24]:
df = merged.drop(['country', 'lat', 'long', 'date', 'date_of_birth', 'nationality'], axis=1)

In [25]:
df.rename(columns={'driver_points': 'driver_points_after', 
    'constructor_points': 'constructor_points_after', 
    'constructor_wins': 'constructor_wins_after', 
    'driver_wins': 'driver_wins_after', 
    'constructor_standings_pos': 'constructor_standings_pos_after',
    'driver_standings_pos': 'driver_standings_pos_after'}, inplace=True)

In [26]:
points_before = {
    'season': [],
    'round': [],
    'driver': [],
    'driver_wins_before': [],
    'driver_points_before': [],
    'driver_standings_before': [],
    'constructor_wins_before': [],
    'constructor_points_before': [],
    'constructor_standings_before': []
}

In [27]:
grouped = df.groupby(['season'])

for season, group in grouped:
    drivers = group.driver.unique()
    for driver in drivers:
        q = group.query('driver ==@driver')
        for index in list(range(len(q))):
            points_before['season'].append(season)
            points_before['round'].append(q.iloc[index]['round'])
            points_before['driver'].append(driver)
            n = q.iloc[index-1]
            if index != 0:
                points_before['driver_points_before'].append(n.driver_points_after)
                points_before['driver_wins_before'].append(n.driver_wins_after)
                points_before['driver_standings_before'].append(n.driver_standings_pos_after)
                points_before['constructor_points_before'].append(n.constructor_points_after)
                points_before['constructor_standings_before'].append(n.constructor_standings_pos_after)
                points_before['constructor_wins_before'].append(n.constructor_wins_after)
            else:
                points_before['driver_points_before'].append(0.0)
                points_before['driver_wins_before'].append(0)
                points_before['driver_standings_before'].append(1)
                points_before['constructor_points_before'].append(0.0)
                points_before['constructor_standings_before'].append(1)
                points_before['constructor_wins_before'].append(0)


points_before = pd.DataFrame(points_before)
points_before.head()

      driver_wins_after  constructor_wins_after
2036                  0                       0
2053                  0                       0
2071                  0                       0
2100                  0                       0
2109                  0                       0
2128                  0                       0
2151                  0                       0
2173                  0                       0
2190                  0                       1
2207                  0                       1
2237                  0                       2
2249                  0                       2
2272                  0                       0
2294                  0                       0
2311                  0                       0
2337                  0                       0
2350                  0                       0
2371                  0                       0
2398                  0                       0
2404                  0                 

/var/folders/77/g3_0sy3x3y5_z5nv9dmdby380000gn/T/ipykernel_11376/3009877750.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for season, group in grouped:


,season,round,driver,driver_wins_before,driver_points_before,driver_standings_before,constructor_wins_before,constructor_points_before,constructor_standings_before
0,2014,1,nico_rosberg,0,0.0,1,0,0.0,1
1,2014,2,nico_rosberg,1,25.0,1,1,25.0,2
2,2014,3,nico_rosberg,1,43.0,1,2,68.0,1
3,2014,4,nico_rosberg,1,61.0,1,3,111.0,1
4,2014,5,nico_rosberg,1,79.0,1,4,154.0,1


In [28]:
df1 = pd.merge(df, points_before, how='inner', on=['season', 'round', 'driver'])

In [29]:
points_from = {
    'season': [],
    'round': [],
    'driver': [],
    'driver_points_from': [],
    'constructor_points_from': []
}

In [30]:
grouped = df1.groupby(['season'])

for season, group in grouped:
    drivers = group.driver.unique()
    for driver in drivers:
        q = group.query('driver ==@driver')
        for index in list(range(len(q))):
            n = q.iloc[index]
            points_from['season'].append(season)
            points_from['round'].append(q.iloc[index]['round'])
            points_from['driver'].append(driver)
            points_from['driver_points_from'].append(n.driver_points_after - n.driver_points_before)
            points_from['constructor_points_from'].append(n.constructor_points_after - n.constructor_points_before)
            

points_from = pd.DataFrame(points_from)

/var/folders/77/g3_0sy3x3y5_z5nv9dmdby380000gn/T/ipykernel_11376/2836361172.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for season, group in grouped:


In [31]:
df2 = pd.merge(df1, points_from, how='inner', on=['season', 'round', 'driver'])

df2[['season', 'round','driver', 'driver_points_before','driver_points_after', 'driver_points_from']].query('season == 2022 & driver == "sebastian_vettel"')

,season,round,driver,driver_points_before,driver_points_after,driver_points_from
3273,2022,3,sebastian_vettel,0.0,0,0.0
3282,2022,4,sebastian_vettel,0.0,4,4.0
3310,2022,5,sebastian_vettel,4.0,4,0.0
3324,2022,6,sebastian_vettel,4.0,4,0.0
3343,2022,7,sebastian_vettel,4.0,5,1.0
3359,2022,8,sebastian_vettel,5.0,13,8.0
3385,2022,9,sebastian_vettel,13.0,13,0.0
3402,2022,10,sebastian_vettel,13.0,15,2.0
3429,2022,11,sebastian_vettel,15.0,15,0.0
3443,2022,12,sebastian_vettel,15.0,15,0.0


In [32]:
more_points = {
    'season': [],
    'round': [],
    'driver': [],
    'driver_points_per': [],
    'constructor_points_per': [],
    'points_percentage': []
}

In [33]:
grouped = df2.groupby(['season'])

for season, group in grouped:
    drivers = group.driver.unique()
    for driver in drivers:
        q = group.query('driver ==@driver')
        for index in list(range(len(q))):
            n = q.iloc[index]
            more_points['season'].append(season)
            more_points['round'].append(q.iloc[index]['round'])
            more_points['driver'].append(driver)
            more_points['driver_points_per'].append(round(n.driver_points_before / (index + 1), 2))
            more_points['constructor_points_per'].append(round(n.constructor_points_before / (index + 1), 2))
            if n.driver_points_before != 0:
                more_points['points_percentage'].append(round((n.driver_points_before / n.constructor_points_before) * 100, 2))
            else:
                more_points['points_percentage'].append(0.00)
            

more_points = pd.DataFrame(more_points)
more_points.head()

/var/folders/77/g3_0sy3x3y5_z5nv9dmdby380000gn/T/ipykernel_11376/1308303808.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for season, group in grouped:


,season,round,driver,driver_points_per,constructor_points_per,points_percentage
0,2014,1,nico_rosberg,0.00,0.00,0.00
1,2014,2,nico_rosberg,12.50,12.50,100.00
2,2014,3,nico_rosberg,14.33,22.67,63.24
3,2014,4,nico_rosberg,15.25,27.75,54.95
4,2014,5,nico_rosberg,15.80,30.80,51.30


In [34]:
df3 = pd.merge(df2, more_points, how='inner', on=['season', 'round', 'driver'])
df3.query('season == 2021 & constructor == "mercedes"')

,season,round,circuit_id,driver,constructor,podium,driver_points_after,driver_wins_after,driver_standings_pos_after,constructor_points_after,...,driver_points_before,driver_standings_before,constructor_wins_before,constructor_points_before,constructor_standings_before,driver_points_from,constructor_points_from,driver_points_per,constructor_points_per,points_percentage
2782,2021,1,bahrain,lewis_hamilton,mercedes,1,25,1,1,41,...,0.0,1,0,0.0,1,25.0,41.0,0.00,0.00,0.00
2784,2021,1,bahrain,valtteri_bottas,mercedes,3,16,0,3,41,...,0.0,1,0,0.0,1,16.0,41.0,0.00,0.00,0.00
2803,2021,2,imola,lewis_hamilton,mercedes,2,44,1,1,60,...,25.0,1,1,41.0,1,19.0,19.0,12.50,20.50,60.98
2819,2021,2,imola,valtteri_bottas,mercedes,18,16,0,5,60,...,16.0,3,1,41.0,1,0.0,19.0,8.00,20.50,39.02
2822,2021,3,portimao,lewis_hamilton,mercedes,1,69,2,1,101,...,44.0,1,1,60.0,1,25.0,41.0,14.67,20.00,73.33
2824,2021,3,portimao,valtteri_bottas,mercedes,3,32,0,4,101,...,16.0,5,1,60.0,1,16.0,41.0,5.33,20.00,26.67
2842,2021,4,catalunya,lewis_hamilton,mercedes,1,94,3,1,141,...,69.0,1,2,101.0,1,25.0,40.0,17.25,25.25,68.32
2844,2021,4,catalunya,valtteri_bottas,mercedes,3,47,0,3,141,...,32.0,4,2,101.0,1,15.0,40.0,8.00,25.25,31.68
2868,2021,5,monaco,lewis_hamilton,mercedes,7,101,3,2,148,...,94.0,1,3,141.0,1,7.0,7.0,18.80,28.20,66.67
2879,2021,5,monaco,valtteri_bottas,mercedes,19,47,0,4,148,...,47.0,3,3,141.0,1,0.0,7.0,9.40,28.20,33.33


In [35]:
points_last_3 = {
    'season': [],
    'round': [],
    'driver': [],
    'driver_last_3': [],
    'constructor_last_3': []
}

In [36]:
grouped = df3.groupby(['season'])

for season, group in grouped:
    drivers = group.driver.unique()
    for driver in drivers:
        q = group.query('driver ==@driver')
        for index in list(range(len(q))):
            points_last_3['season'].append(season)
            points_last_3['round'].append(q.iloc[index]['round'])
            points_last_3['driver'].append(driver)

            last_3 = list(filter(lambda x : x >= 0, list(range(index-1, index-4, -1))))
            if last_3 == []:
                points_last_3['driver_last_3'].append(0.0)
                points_last_3['constructor_last_3'].append(0.0)
            else:
                n = q.iloc[last_3]
                points_last_3['driver_last_3'].append(n.driver_points_from.sum())
                points_last_3['constructor_last_3'].append(n.constructor_points_from.sum())
                

points_last_3 = pd.DataFrame(points_last_3)

/var/folders/77/g3_0sy3x3y5_z5nv9dmdby380000gn/T/ipykernel_11376/3230430731.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for season, group in grouped:


In [37]:
df4 = pd.merge(df3, points_last_3, how='inner', on=['season', 'round', 'driver'])
df4.query('season == 2021 & driver == "lewis_hamilton"')

,season,round,circuit_id,driver,constructor,podium,driver_points_after,driver_wins_after,driver_standings_pos_after,constructor_points_after,...,constructor_wins_before,constructor_points_before,constructor_standings_before,driver_points_from,constructor_points_from,driver_points_per,constructor_points_per,points_percentage,driver_last_3,constructor_last_3
2782,2021,1,bahrain,lewis_hamilton,mercedes,1,25,1,1,41,...,0,0.0,1,25.0,41.0,0.00,0.00,0.00,0.0,0.0
2803,2021,2,imola,lewis_hamilton,mercedes,2,44,1,1,60,...,1,41.0,1,19.0,19.0,12.50,20.50,60.98,25.0,41.0
2822,2021,3,portimao,lewis_hamilton,mercedes,1,69,2,1,101,...,1,60.0,1,25.0,41.0,14.67,20.00,73.33,44.0,60.0
2842,2021,4,catalunya,lewis_hamilton,mercedes,1,94,3,1,141,...,2,101.0,1,25.0,40.0,17.25,25.25,68.32,69.0,101.0
2868,2021,5,monaco,lewis_hamilton,mercedes,7,101,3,2,148,...,3,141.0,1,7.0,7.0,18.80,28.20,66.67,69.0,100.0
2895,2021,6,baku,lewis_hamilton,mercedes,15,101,3,2,148,...,3,148.0,2,0.0,0.0,16.83,24.67,68.24,57.0,88.0
2902,2021,7,ricard,lewis_hamilton,mercedes,2,119,3,2,178,...,3,148.0,2,18.0,30.0,14.43,21.14,68.24,32.0,47.0
2921,2021,8,red_bull_ring,lewis_hamilton,mercedes,2,138,3,2,212,...,3,178.0,2,19.0,34.0,14.88,22.25,66.85,25.0,37.0
2943,2021,9,red_bull_ring,lewis_hamilton,mercedes,4,150,3,2,242,...,3,212.0,2,12.0,30.0,15.33,23.56,65.09,37.0,64.0
2960,2021,10,silverstone,lewis_hamilton,mercedes,1,177,4,2,285,...,3,242.0,2,27.0,43.0,15.00,24.20,61.98,49.0,94.0


In [38]:
## Removing sprint races

df5 = df4[~
(
    ((df4.season == 2014) & (df4['round'] == 19))
    |
    ((df4.season == 2021) & (df4['round'].isin([10, 14, 19])))
    |
    ((df4.season == 2022) & (df4['round'].isin([4, 11, 21])))
)
]

In [39]:
df6 = df5.drop([
    'constructor_wins_after', 
    'constructor_points_after', 
    'constructor_points_from',
    'driver_points_after', 
    'driver_wins_after', 
    'driver_standings_pos_after',
    'constructor_standings_pos_after'
    ], axis=1)

In [40]:
df6.head()

,season,round,circuit_id,driver,constructor,podium,qualifying_pos,stage,q_delta,starting_grid,...,driver_standings_before,constructor_wins_before,constructor_points_before,constructor_standings_before,driver_points_from,driver_points_per,constructor_points_per,points_percentage,driver_last_3,constructor_last_3
0,2014,1,albert_park,nico_rosberg,mercedes,1,3,q3,0.000,3,...,1,0,0.0,1,25.0,0.0,0.0,0.0,0.0,0.0
1,2014,1,albert_park,kevin_magnussen,mclaren,2,4,q3,0.000,4,...,1,0,0.0,1,18.0,0.0,0.0,0.0,0.0,0.0
2,2014,1,albert_park,jenson_button,mclaren,3,11,q2,0.206,10,...,1,0,0.0,1,15.0,0.0,0.0,0.0,0.0,0.0
3,2014,1,albert_park,fernando_alonso,ferrari,4,5,q3,0.000,5,...,1,0,0.0,1,12.0,0.0,0.0,0.0,0.0,0.0
4,2014,1,albert_park,valtteri_bottas,williams,5,10,q3,3.916,15,...,1,0,0.0,1,10.0,0.0,0.0,0.0,0.0,0.0


In [41]:
df6.to_csv(path+'data/processed.csv', index=False)